In [1]:
from misophonia_dataset._log import setup_print_logging

setup_print_logging()

import pandas as pd

from misophonia_dataset.source_data.esc50 import Esc50Dataset
from misophonia_dataset.source_data.fsd50k import Fsd50kDataset
from misophonia_dataset.source_data.foams import FoamsDataset

# Loading individual source data

In [2]:
esc50 = Esc50Dataset()
esc50.download_data()
esc50_meta = esc50.get_metadata()

esc50_meta.head()

,esc50_filename,esc50_fold,esc50_target,esc50_category,esc50_esc10,freesound_id,esc50_take,source_dataset,file_path,labels,label_type,licensing,split
30,1-12653-A-15.wav,1,15,water_drops,False,12653,A,ESC50,/workspaces/misophonia-dataset/data/ESC50/ESC-...,[water_drops],trigger,({'license_url': 'http://creativecommons.org/l...,train
31,1-12654-A-15.wav,1,15,water_drops,False,12654,A,ESC50,/workspaces/misophonia-dataset/data/ESC50/ESC-...,[water_drops],trigger,({'license_url': 'http://creativecommons.org/l...,train
32,1-12654-B-15.wav,1,15,water_drops,False,12654,B,ESC50,/workspaces/misophonia-dataset/data/ESC50/ESC-...,[water_drops],trigger,({'license_url': 'http://creativecommons.org/l...,train
36,1-137-A-32.wav,1,32,keyboard_typing,False,137,A,ESC50,/workspaces/misophonia-dataset/data/ESC50/ESC-...,[typing],trigger,({'license_url': 'https://creativecommons.org/...,train
50,1-16746-A-15.wav,1,15,water_drops,False,16746,A,ESC50,/workspaces/misophonia-dataset/data/ESC50/ESC-...,[water_drops],trigger,({'license_url': 'https://creativecommons.org/...,val


In [3]:
fsd50k = Fsd50kDataset()
fsd50k.download_data()
fsd50k_meta = fsd50k.get_metadata()

assert fsd50k_meta.loc[fsd50k_meta["fsd50k_split"] == "eval", "split"].isin(["test"]).all(), (
    "All samples from FSD50K 'eval' must be in the 'test' split"
)

fsd50k_meta.head()

,fsd50k_fname,fsd50k_labels,fsd50k_mids,fsd50k_split,source_dataset,freesound_id,file_path,label_type,labels,licensing,split
0,64760,[Electric_guitar],/m/02sgy,dev,FSD50K,64760,/workspaces/misophonia-dataset/data/FSD50K/dev...,background,[Electric_guitar],({'license_url': 'http://creativecommons.org/l...,train
1,16399,[Electric_guitar],/m/02sgy,dev,FSD50K,16399,/workspaces/misophonia-dataset/data/FSD50K/dev...,background,[Electric_guitar],({'license_url': 'http://creativecommons.org/l...,train
2,16401,[Electric_guitar],/m/02sgy,dev,FSD50K,16401,/workspaces/misophonia-dataset/data/FSD50K/dev...,background,[Electric_guitar],({'license_url': 'http://creativecommons.org/l...,train
3,16402,[Electric_guitar],/m/02sgy,dev,FSD50K,16402,/workspaces/misophonia-dataset/data/FSD50K/dev...,background,[Electric_guitar],({'license_url': 'http://creativecommons.org/l...,train
4,16404,[Electric_guitar],/m/02sgy,dev,FSD50K,16404,/workspaces/misophonia-dataset/data/FSD50K/dev...,background,[Electric_guitar],({'license_url': 'http://creativecommons.org/l...,train


In [4]:
foams = FoamsDataset()
foams.download_data()
foams_meta = foams.get_metadata()

assert foams_meta["split"].nunique() == 1 and foams_meta["split"].unique()[0] == "test", (
    "All FOAMS samples must be in the 'test' split"
)

foams_meta.head()

,freesound_id,labels,foams_duration,source_dataset,label_type,file_path,licensing,split
0,184320,[basketball_dribbling],3.270003,FOAMS,trigger,/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...,test
1,580220,[basketball_dribbling],3.862957,FOAMS,trigger,/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...,test
2,262507,[basketball_dribbling],4.903607,FOAMS,trigger,/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...,test
3,453757,[basketball_dribbling],5.097981,FOAMS,trigger,/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...,test
4,530362,[basketball_dribbling],5.344224,FOAMS,trigger,/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...,test


# Combining all source data

In [5]:
combined_meta = pd.concat([esc50_meta, fsd50k_meta, foams_meta], ignore_index=True)
combined_meta = combined_meta[
    # These colums are there, see the interface.SourceMetaData class
    ["source_dataset", "split", "freesound_id", "label_type", "labels", "file_path", "licensing"] 
]

## Examine different subsets

In [6]:
def value_counts_pcts(col: pd.Series) -> pd.DataFrame:
    normalized = col.value_counts(normalize=True)
    counts = col.value_counts()
    return pd.DataFrame({"count": counts, "percentage": normalized * 100}, index=counts.index)

def print_subset_info(subset_descr: str, subset: pd.DataFrame):
    print(f"\n=== Subset: {subset_descr} ===")
    print(f"* Total samples: {len(subset)}")
    print("\n* Distribution by source dataset:")
    print(value_counts_pcts(subset["source_dataset"]))
    print(f"\n* Distribution by split:")
    print(value_counts_pcts(subset["split"]))
    print(f"\n* Distribution by label type:")
    print(value_counts_pcts(subset["label_type"]))
    print("\n* Sample entries:")
    display(subset.sample(min(3, len(subset))))

### Examine all source data

In [7]:
print_subset_info("All source data", combined_meta)


=== Subset: All source data ===
* Total samples: 31951

* Distribution by source dataset:
                count  percentage
source_dataset                   
FSD50K          31701   99.217552
ESC50             200    0.625958
FOAMS              50    0.156490

* Distribution by split:
       count  percentage
split                   
train  23993   75.093111
test    4697   14.700635
val     3261   10.206253

* Distribution by label type:
            count  percentage
label_type                   
background  28640   89.637257
trigger      2417    7.564708
control       894    2.798034

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
17100,FSD50K,train,380045,background,[Choir],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
11702,FSD50K,val,142863,background,[Snare_drum],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
29703,FSD50K,test,381275,background,"[Belly_laugh, Giggle]",/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/p...


### Examine by label type

In [8]:
# For each label type
for label_type in combined_meta["label_type"].unique():
    subset = combined_meta[combined_meta["label_type"] == label_type]
    print_subset_info(f"label_type = {label_type}", subset)


=== Subset: label_type = trigger ===
* Total samples: 2417

* Distribution by source dataset:
                count  percentage
source_dataset                   
FSD50K           2167   89.656599
ESC50             200    8.274721
FOAMS              50    2.068680

* Distribution by split:
       count  percentage
split                   
train   1646   68.100952
test     548   22.672735
val      223    9.226314

* Distribution by label type:
            count  percentage
label_type                   
trigger      2417       100.0

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
28952,FSD50K,test,243971,trigger,[water_drops],/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/l...
31002,FSD50K,test,44297,trigger,[typing],/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/p...
10684,FSD50K,train,336418,trigger,[water_drops],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/p...



=== Subset: label_type = background ===
* Total samples: 28640

* Distribution by source dataset:
                count  percentage
source_dataset                   
FSD50K          28640       100.0

* Distribution by split:
       count  percentage
split                   
train  21672   75.670391
test    4021   14.039804
val     2947   10.289804

* Distribution by label type:
            count  percentage
label_type                   
background  28640       100.0

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
4418,FSD50K,train,99767,background,"[Dog, Bird]",/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
9109,FSD50K,train,103601,background,[Walk_and_footsteps],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
21236,FSD50K,train,180686,background,[Zipper_(clothing)],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/p...



=== Subset: label_type = control ===
* Total samples: 894

* Distribution by source dataset:
                count  percentage
source_dataset                   
FSD50K            894       100.0

* Distribution by split:
       count  percentage
split                   
train    675   75.503356
test     128   14.317673
val       91   10.178971

* Distribution by label type:
            count  percentage
label_type                   
control       894       100.0

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
14565,FSD50K,train,85335,control,[piano],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
14412,FSD50K,train,148485,control,[piano],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
22930,FSD50K,train,72945,control,[harp],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...


### Examine by trigger category

In [9]:
# For each trigger sound label
for label in combined_meta[combined_meta["label_type"] == "trigger"]["labels"].explode().unique():
    subset = combined_meta[
        (combined_meta["label_type"] == "trigger") & (combined_meta["labels"].apply(lambda labels: label in labels))
    ]
    print_subset_info(f"Trigger = {label}", subset)


=== Subset: Trigger = water_drops ===
* Total samples: 328

* Distribution by source dataset:
                count  percentage
source_dataset                   
FSD50K            283   86.280488
ESC50              40   12.195122
FOAMS               5    1.524390

* Distribution by split:
       count  percentage
split                   
train    230   70.121951
test      63   19.207317
val       35   10.670732

* Distribution by label type:
            count  percentage
label_type                   
trigger       328       100.0

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
30004,FSD50K,test,176753,trigger,[water_drops],/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/p...
10694,FSD50K,train,273998,trigger,[water_drops],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
30012,FSD50K,test,209,trigger,[water_drops],/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/l...



=== Subset: Trigger = typing ===
* Total samples: 434

* Distribution by source dataset:
                count  percentage
source_dataset                   
FSD50K            389   89.631336
ESC50              40    9.216590
FOAMS               5    1.152074

* Distribution by split:
       count  percentage
split                   
train    282   64.976959
test     123   28.341014
val       29    6.682028

* Distribution by label type:
            count  percentage
label_type                   
trigger       434       100.0

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
31028,FSD50K,test,256290,trigger,[typing],/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/l...
30982,FSD50K,test,52051,trigger,[typing],/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/p...
31036,FSD50K,test,84393,trigger,[typing],/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/l...



=== Subset: Trigger = swallowing ===
* Total samples: 110

* Distribution by source dataset:
                count  percentage
source_dataset                   
FSD50K             65   59.090909
ESC50              40   36.363636
FOAMS               5    4.545455

* Distribution by split:
       count  percentage
split                   
train     76   69.090909
test      18   16.363636
val       16   14.545455

* Distribution by label type:
            count  percentage
label_type                   
trigger       110       100.0

* Sample entries:


...

,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
30881,FSD50K,test,268143,trigger,[knife_cutting],/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/l...
27155,FSD50K,val,66679,trigger,[knife_cutting],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
22367,FSD50K,train,237317,trigger,[knife_cutting],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/p...



=== Subset: Trigger = basketball_dribbling ===
* Total samples: 5

* Distribution by source dataset:
                count  percentage
source_dataset                   
FOAMS               5       100.0

* Distribution by split:
       count  percentage
split                   
test       5       100.0

* Distribution by label type:
            count  percentage
label_type                   
trigger         5       100.0

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
31905,FOAMS,test,530362,trigger,[basketball_dribbling],/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...
31902,FOAMS,test,580220,trigger,[basketball_dribbling],/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...
31904,FOAMS,test,453757,trigger,[basketball_dribbling],/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...



=== Subset: Trigger = flipping_newspaper ===
* Total samples: 5

* Distribution by source dataset:
                count  percentage
source_dataset                   
FOAMS               5       100.0

* Distribution by split:
       count  percentage
split                   
test       5       100.0

* Distribution by label type:
            count  percentage
label_type                   
trigger         5       100.0

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
31918,FOAMS,test,484956,trigger,[flipping_newspaper],/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...
31916,FOAMS,test,636467,trigger,[flipping_newspaper],/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...
31919,FOAMS,test,557540,trigger,[flipping_newspaper],/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...


### Examine by source dataset

In [10]:
# for each source dataset
for source_dataset in combined_meta["source_dataset"].unique():
    subset = combined_meta[combined_meta["source_dataset"] == source_dataset]
    print_subset_info(f"Source dataset = {source_dataset}", subset)


=== Subset: Source dataset = ESC50 ===
* Total samples: 200

* Distribution by source dataset:
                count  percentage
source_dataset                   
ESC50             200       100.0

* Distribution by split:
       count  percentage
split                   
train    131        65.5
test      53        26.5
val       16         8.0

* Distribution by label type:
            count  percentage
label_type                   
trigger       200       100.0

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
84,ESC50,val,125548,trigger,[typing],/workspaces/misophonia-dataset/data/ESC50/ESC-...,({'license_url': 'http://creativecommons.org/p...
124,ESC50,train,157296,trigger,[clearing_throat],/workspaces/misophonia-dataset/data/ESC50/ESC-...,({'license_url': 'http://creativecommons.org/p...
135,ESC50,train,177243,trigger,[typing],/workspaces/misophonia-dataset/data/ESC50/ESC-...,({'license_url': 'https://creativecommons.org/...



=== Subset: Source dataset = FSD50K ===
* Total samples: 31701

* Distribution by source dataset:
                count  percentage
source_dataset                   
FSD50K          31701       100.0

* Distribution by split:
       count  percentage
split                   
train  23862   75.272073
test    4594   14.491656
val     3245   10.236270

* Distribution by label type:
            count  percentage
label_type                   
background  28640   90.344153
trigger      2167    6.835747
control       894    2.820100

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
22361,FSD50K,train,415483,background,[Car_passing_by],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
22340,FSD50K,train,425850,background,[Car_passing_by],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
27930,FSD50K,test,217800,background,"[Wind_chime, Car_passing_by, Wind]",/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/p...



=== Subset: Source dataset = FOAMS ===
* Total samples: 50

* Distribution by source dataset:
                count  percentage
source_dataset                   
FOAMS              50       100.0

* Distribution by split:
       count  percentage
split                   
test      50       100.0

* Distribution by label type:
            count  percentage
label_type                   
trigger        50       100.0

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
31914,FOAMS,test,586979,trigger,[clearing_throat],/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...
31929,FOAMS,test,419998,trigger,[knife_cutting],/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...
31937,FOAMS,test,267932,trigger,[swallowing],/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...


### Examine single vs. multi-label data

In [11]:
def normalize_label_tuple(s: pd.Series) -> pd.Series:
    return s.apply(lambda labels: tuple(sorted(labels)))

label_len = combined_meta["labels"].apply(len)
for label_count in sorted(label_len.unique()):
    subset = combined_meta[label_len == label_count]
    print_subset_info(f"Number of labels = {label_count}", subset)
    print(f"\n* Label pairs distribution:")
    print(value_counts_pcts(normalize_label_tuple(subset["labels"])))

    print("\n ---- ")


=== Subset: Number of labels = 1 ===
* Total samples: 28752

* Distribution by source dataset:
                count  percentage
source_dataset                   
FSD50K          28502   99.130495
ESC50             200    0.695604
FOAMS              50    0.173901

* Distribution by split:
       count  percentage
split                   
train  22145   77.020729
test    3610   12.555648
val     2997   10.423623

* Distribution by label type:
            count  percentage
label_type                   
background  25469   88.581664
trigger      2389    8.308987
control       894    3.109349

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
22905,FSD50K,train,415447,control,[harp],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/p...
20312,FSD50K,train,56083,background,[Violin_and_fiddle],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/p...
13178,FSD50K,train,48608,background,[Drum_kit],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...



* Label pairs distribution:
                              count  percentage
labels                                         
(Laughter,)                     691    2.403311
(Snare_drum,)                   684    2.378965
(Cello,)                        660    2.295492
(Trumpet,)                      610    2.121592
(Clarinet,)                     592    2.058987
...                             ...         ...
(Yodeling,)                       2    0.006956
(Mouse,)                          1    0.003478
(Sailboat_and_sailing_ship,)      1    0.003478
(Rail_transport,)                 1    0.003478
(Music,)                          1    0.003478

[195 rows x 2 columns]

 ---- 

=== Subset: Number of labels = 2 ===
* Total samples: 2737

* Distribution by source dataset:
                count  percentage
source_dataset                   
FSD50K           2737       100.0

* Distribution by split:
       count  percentage
split                   
train   1668   60.942638
test     837   30

,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
2610,FSD50K,train,70363,background,"[Door, Cupboard_open_or_close]",/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
5497,FSD50K,train,24332,background,"[Motorcycle, Engine]",/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
15913,FSD50K,train,252489,background,"[Acoustic_guitar, Human_voice]",/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...



* Label pairs distribution:
                                                   count  percentage
labels                                                              
(Sink_(filling_or_washing), Water_tap_and_faucet)    123    4.493972
(Splash_and_splatter, Water)                         112    4.092072
(Drum, Drum_kit)                                      74    2.703690
(Applause, Cheering)                                  55    2.009499
(Applause, Crowd)                                     41    1.497991
...                                                  ...         ...
(Glass, Keys_jangling)                                 1    0.036536
(Glass, Hammer)                                        1    0.036536
(Glass, Singing)                                       1    0.036536
(Accordion, Plucked_string_instrument)                 1    0.036536
(Choir, Organ)                                         1    0.036536

[646 rows x 2 columns]

 ---- 

=== Subset: Number of labels = 3 ===
* To

...


* Label pairs distribution:
                                                    count  percentage
labels                                                               
(Bass_drum, Drum_kit, Hi-hat, Snare_drum, Synth...      3       18.75
(Bus, Coin_(dropping), Conversation, Walk_and_f...      1        6.25
(Bass_drum, Electric_guitar, Hi-hat, Snare_drum...      1        6.25
(Drum, Drum_kit, Electric_guitar, Hi-hat, Tambo...      1        6.25
(Bird_flight_and_flapping_wings, Duck, Goose, S...      1        6.25
(Conversation, Female_speech_and_woman_speaking...      1        6.25
(Bass_drum, Hi-hat, Organ, Snare_drum, Synthesi...      1        6.25
(Bass_drum, Cymbal, Drum_kit, Snare_drum, Synth...      1        6.25
(Bass_drum, Organ, Snare_drum, Synthesizer, Tam...      1        6.25
(Bass_guitar, Hi-hat, Snare_drum, Synthesizer, ...      1        6.25
(Bass_drum, Bell, Hi-hat, Snare_drum, Synthesizer)      1        6.25
(Drum, French_horn, Orchestra, Tambourine, Viol...      1    

,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
27404,FSD50K,test,187299,background,"[Electric_guitar, Bass_drum, Alto_saxophone, B...",/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/l...



* Label pairs distribution:
                                                    count  percentage
labels                                                               
(Alto_saxophone, Bass_drum, Bass_guitar, Electr...      1       100.0

 ---- 

=== Subset: Number of labels = 7 ===
* Total samples: 2

* Distribution by source dataset:
                count  percentage
source_dataset                   
FSD50K              2       100.0

* Distribution by split:
       count  percentage
split                   
test       2       100.0

* Distribution by label type:
            count  percentage
label_type                   
background      2       100.0

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
28494,FSD50K,test,431330,background,"[Drum, Wind_instrument_and_woodwind_instrument...",/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/l...
28254,FSD50K,test,368610,background,"[Bass_drum, Snare_drum, Synthesizer, Cowbell, ...",/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/l...



* Label pairs distribution:
                                                    count  percentage
labels                                                               
(Bass_drum, Cowbell, Hi-hat, Snare_drum, Synthe...      1        50.0
(Brass_instrument, Cello, Double_bass, Drum, Or...      1        50.0

 ---- 


### Examine single vs. multi-label trigger data

In [12]:
trigger_subset = combined_meta[combined_meta["label_type"] == "trigger"]
label_len = trigger_subset["labels"].apply(len)
for label_count in sorted(label_len.unique()):
    subset = trigger_subset[label_len == label_count]
    print_subset_info(f"Number of labels = {label_count}", subset)
    print(f"\n* Label pairs distribution:")
    print(value_counts_pcts(normalize_label_tuple(subset["labels"])))

    print("\n ---- ")


=== Subset: Number of labels = 1 ===
* Total samples: 2389

* Distribution by source dataset:
                count  percentage
source_dataset                   
FSD50K           2139   89.535370
ESC50             200    8.371704
FOAMS              50    2.092926

* Distribution by split:
       count  percentage
split                   
train   1641   68.689828
test     525   21.975722
val      223    9.334450

* Distribution by label type:
            count  percentage
label_type                   
trigger      2389       100.0

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
2,ESC50,train,12654,trigger,[water_drops],/workspaces/misophonia-dataset/data/ESC50/ESC-...,({'license_url': 'http://creativecommons.org/l...
30993,FSD50K,test,335020,trigger,[typing],/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/l...
22395,FSD50K,train,426819,trigger,[knife_cutting],/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/p...



* Label pairs distribution:
                         count  percentage
labels                                    
(typing,)                  434   18.166597
(plastic_crumpling,)       429   17.957304
(human_breathing,)         352   14.734198
(clearing_throat,)         337   14.106321
(water_drops,)             326   13.645877
(chewing_gum,)             205    8.580996
(knife_cutting,)           188    7.869401
(swallowing,)              108    4.520720
(basketball_dribbling,)      5    0.209293
(flipping_newspaper,)        5    0.209293

 ---- 

=== Subset: Number of labels = 2 ===
* Total samples: 28

* Distribution by source dataset:
                count  percentage
source_dataset                   
FSD50K             28       100.0

* Distribution by split:
       count  percentage
split                   
test      23   82.142857
train      5   17.857143

* Distribution by label type:
            count  percentage
label_type                   
trigger        28       100.0

* Sa

,source_dataset,split,freesound_id,label_type,labels,file_path,licensing
29961,FSD50K,test,85292,trigger,"[clearing_throat, human_breathing]",/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'https://creativecommons.org/...
16721,FSD50K,train,73327,trigger,"[clearing_throat, chewing_gum]",/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
29933,FSD50K,test,408107,trigger,"[clearing_throat, human_breathing]",/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/l...



* Label pairs distribution:
                                    count  percentage
labels                                               
(clearing_throat, human_breathing)     18   64.285714
(chewing_gum, human_breathing)          4   14.285714
(chewing_gum, clearing_throat)          2    7.142857
(swallowing, water_drops)               2    7.142857
(chewing_gum, plastic_crumpling)        2    7.142857

 ---- 
